In [1]:
import pandas as pd
import numpy as np
from PIL import Image 
import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import cv2

In [2]:
Thickness = pd.read_csv('./data/Thickness.csv')
Thickness = Thickness.drop([33]) # 9-1 drop
Thickness.reset_index(drop=True, inplace=True)
Thickness

,Thickness
0,48.266667
1,47.966667
2,48.166667
3,48.600000
4,59.533333
...,...
122,84.633333
123,98.366667
124,97.033333
125,96.266667


In [3]:
img_path_list = []
Top_path = './data/box_img/Top/'
Bot_path = './data/box_img/Bot/'
Top_change_VS_path = './data/box_img/Top_change_VS/'

for i in range(1, 33):
    for j in range(1, 5):
        if i == 9 and j == 1: 
            continue
        img_path_list.append('./data/test_img/AI-{}_0{}_transformed.jpg'.format(i, j))

In [4]:
# 첫번째 bot 이미지를 기준 이미지로 설정
reference_bot = cv2.cvtColor(np.array(Image.open(Bot_path + 'Bot_0.jpg')), cv2.COLOR_RGB2BGR)
reference_bot_hsv = cv2.cvtColor(reference_bot, cv2.COLOR_BGR2HSV)

In [5]:
# 폴더가 존재하지 않으면 생성
if not os.path.exists(Top_change_VS_path):
    os.makedirs(Top_change_VS_path)

# 데이터 프레임을 위한 리스트 초기화
data = []

for i in range(len(img_path_list)):
    top = cv2.cvtColor(np.array(Image.open(Top_path + 'Top_{}.jpg'.format(i))), cv2.COLOR_RGB2BGR)
    bot = cv2.cvtColor(np.array(Image.open(Bot_path + 'Bot_{}.jpg'.format(i))), cv2.COLOR_RGB2BGR)

    # 명도와 채도의 변화를 백분율로 계산
    bot_hsv = cv2.cvtColor(bot, cv2.COLOR_BGR2HSV)
    delta_V = bot_hsv[:,:,2].mean() / reference_bot_hsv[:,:,2].mean()
    delta_S = bot_hsv[:,:,1].mean() / reference_bot_hsv[:,:,1].mean()

    # 계산한 변화를 top 이미지에 적용
    top_hsv = cv2.cvtColor(top, cv2.COLOR_BGR2HSV)
    top_hsv[:,:,2] = np.clip(top_hsv[:,:,2] + delta_V, 0, 255)
    top_hsv[:,:,1] = np.clip(top_hsv[:,:,1] + delta_S, 0, 255)
    top_changed = cv2.cvtColor(top_hsv, cv2.COLOR_HSV2BGR)
    
    # 변화시킨 이미지를 저장
    Image.fromarray(cv2.cvtColor(top_changed, cv2.COLOR_BGR2RGB)).save(os.path.join(Top_change_VS_path, 'Top_change_VS_{}.jpg'.format(i)))



vs에 대해 전처리한 데이터

In [6]:
top_avg_bgr = []
top_avg_hsv = []

for i in range(len(img_path_list)):
    # 이미지 로드
    img = cv2.cvtColor(np.array(Image.open(Top_change_VS_path + 'Top_change_VS_{}.jpg'.format(i))), cv2.COLOR_RGB2BGR)
    # RGB 평균 계산
    avg_bgr = img.mean(axis=(0,1))
    top_avg_bgr.append(avg_bgr)
    # HSV 평균 계산
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    avg_hsv = img_hsv.mean(axis=(0,1))
    top_avg_hsv.append(avg_hsv)

In [7]:
Table_dataset = pd.DataFrame({'Top_R' : [i[2] for i in top_avg_bgr], 
                              'Top_G' : [i[1] for i in top_avg_bgr], 
                              'Top_B' : [i[0] for i in top_avg_bgr],
                              'Top_H' : [i[0] for i in top_avg_hsv], 
                              'Top_S' : [i[1] for i in top_avg_hsv], 
                              'Top_V' : [i[2] for i in top_avg_hsv],
                              'Thickness': Thickness['Thickness']})

Table_dataset

,Top_R,Top_G,Top_B,Top_H,Top_S,Top_V,Thickness
0,12.1656,11.7032,8.1244,23.5788,90.0008,12.3960,48.266667
1,12.4724,12.7284,11.0100,24.1728,39.9828,12.8452,47.966667
2,12.6808,12.6408,9.8016,29.8416,62.2292,12.8824,48.166667
3,11.1372,11.4652,8.5556,27.2936,68.9256,11.6236,48.600000
4,5.4440,3.1864,1.4096,30.2408,206.5236,5.5604,59.533333
...,...,...,...,...,...,...,...
122,0.4860,4.5392,22.1104,114.2120,249.3836,22.1104,84.633333
123,0.7240,15.4616,33.4912,106.3536,249.7260,33.4912,98.366667
124,0.9048,13.1052,38.7656,110.3592,249.1856,38.7656,97.033333
125,1.0604,13.6216,35.2664,108.8924,247.8428,35.2664,96.266667


# **이미지 0에 대해선 변하면 안되는데 변함**

In [ ]:
Table_dataset.to_csv('./data/Top_change_VS_path.csv', index=False)
